# Generating 26 cancer gene regulatory network using TCGA datasets
Authors: Camila Lopes-Ramos<sup>1</sup>

<sup>1</sup> Department of Biostatistics, Harvard T.H. Chan School of Public Health, Boston, MA.

## Introduction

In this tutorial, we will generate 26 cancer gene regulatory networks using PANDA<sup>1</sup> from TCGA<sup>5</sup> gene expression data. First, we need to normalize gene expression data accounting for batch effects using smooth quantile normalization method (qsmooth)<sup>2</sup>, then we will call PANDA by using gene expression for each tissue as input along with TF PPI from StringDB<sup>3</sup>, and a motif prior network using TF motifs from CIS-BP ran on promoter regions of target genes using FIMO<sup>4</sup>.

The generated networks are available in the cancer catalog of [GRAND](https://grand.networkmedicine.org/)<sup>6</sup> database. To start, we need to set the path to the input data files on the server.

In [ ]:
ppath='/opt/data/netZooR/tcganets/'

Some parts of this analysis can be time-consuming, therefore, we set the following parameter `precomputed` to load precomputed data to accelerate the analysis.

In [ ]:
precomputed=1

Then, we load the libraries for this analysis.

In [ ]:
library(Biobase) # To build expr objects
library(qsmooth) # For quantile normalization
library(netZooR) # For GRN reconstruction using PANDA

## 1. Normalize gene expression data using qsmooth

The first part consists of normalizing gene expression data using Qsmooth<sup>2</sup>. This method allows to normalize gene expression data within groups (cancer types), however, it allows for variation between groups. Normalizing across groups without taking into account the group structure as implemented in Qsmooth might remove important biological variability between groups. For this section, we start first by loading gene expression and clinical data for 26 cancer types from TCGA.

In [ ]:
load(paste0(ppath,"rnaseq_tumor_clin2019_06_26.rdata"))

Next, we extract gene expression data counts from the loaded data using `exprs` function from the `biobase` package.

In [ ]:
counts <- exprs(obj2)

The counts matrix has 60,483 counts for genes and transcripts across 6,844 cancer samples. Next, we normalize the counts within cancer types while maintaining between-group biological variability using Qsmooth. Qsmooth takes as input the counts matrix and a group factor which corresponds to the membership of each sample to a cancer type.

In [ ]:
if(precomputed==0){
    qsmooth1 <- qsmooth(object = counts, group_factor = pData(obj2)$tumorID)
    qs <- qsmoothData(qsmooth1)
}else if (precomputed==1){
    qs<-readRDS(paste0(ppath,"qs.rds"))  
}

Finally, we log transform this matrix as follows:

In [ ]:
exp <- log2(qs+1)
exp

The normalized gene expression matrix has the same number of samples and genes, with gene expression transformed to continuous variables as opposed to counts in the original matrix. In the next step, we will save the gene expression for each cancer type separately, which will allow us to build a PANDA gene regualtory network for each cancer type. We can also save the phenotypic variables for each sample, and we set a network folder to save the final networks. To do so, we need to create specific folders on the local disk space. 

In [ ]:
dir.create('../data/expression')
dir.create('../data/phenotypic')
dir.create('../data/network')

The cancer types included have the following abbreviations, which are described in [TCGA](https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/tcga-study-abbreviations).

In [ ]:
cancer <- unique(pData(obj2)$tumorID)
cancer

In addition, the clinical variables we might be interested for follow-up analysis, such as sex differences in cancer and regulatory biomarkers of survival, are the following:

In [ ]:
j <- c("age_at_initial_pathologic_diagnosis", "gender", "race", "ajcc_pathologic_tumor_stage", "days_to_last_followup", "vital_status", "tumorID")

Finally, we iterate over cancer types to save the gene expression and phenotypic information as separate files.

In [ ]:
if(precomputed==0){
    for(i in 1:length(cancer)){
      k <- which(pData(obj2)$tumorID == cancer[i])
      exp1 <- exp[,k]
      clin <- pData(obj2)[k,]
      d <- ifelse(is.na(clin$days_to_last_followup), yes=clin$last_contact_days_to, no=clin$days_to_last_followup)
      clin$days_to_last_followup <- d
      clin1 <- clin[,which(colnames(clin) %in% j)]
      #save the files
      write.table(exp1, file=paste0("../data/expression/expression_tcga_", cancer[i],".txt"),sep="\t",quote=F)
      write.table(clin1,file=paste0("../data/phenotypic/pheno_tcga_", cancer[i],".txt"),sep="\t",quote=F)
    }
}

For example, we can load the gene expression file of Uveal Melanoma (UVM).

In [ ]:
exp1 <- read.table(paste0(ppath,'expression/expression_tcga_UVM.txt'))

The dataframe has normalized gene expression data for 60,483 genes and transcripts across 80 UVM donors.

## 2. Gene regulatory network reconstruction
Now, we will generate 26 cancer gene regulatory networks using the data processed above and PANDA<sup>1</sup> as implemented in [netZooR](netzoo.github.io) v0.9. PANDA<sup>1</sup> takes as input three networks to builg a gene regulatory network. Two of these inputs are "generic", meaning that they do not carry information about context. These networks are motif network, that determines the presence of a TF motif in the promoter region of a target gene.

In [ ]:
motif <- read.delim(paste0(ppath,"motif.txt"),stringsAsFactors=F,header=F)
motif

The motif network has 3 columns: a TF column, a target gene column, and a weight column that has 1 if a TF has a motif in the promoter region of the target gene. The second generic network is a TF PPI network that determines which TFs act together to form transcriptional complexes and together regulat their target genes.

In [ ]:
ppi <- read.delim(paste0(ppath,"ppi.txt"),stringsAsFactors=F,header=F)
ppi

The PPI network has three columns, the first TF in the interacting pair, a second TF, and a weight column that varies between 0 and 1 and denotes the strength of interaction between TFs.

The third network is context-specific, and that the gene co-expression network built from the gene expression data for each cancer type. The final PANDA networks are complete bipartite graphs linking TFs to their target genes. Now, for each cancer type, we call PANDA using these three inputs and with the `mode` set to `intersection`, which will take the intersecting set of TFs and genes across all networks. We saw previously that gene expression data had more than 60,000 genes and transcript, by setting the intersction mode, we will restrict the genes to the ones present in the motif network, which includes protein-coding genes.

In [ ]:
if(precomputed==0){
    for (cancer in c("ACC","BLCA","CHOL","COAD","DLBC","ESCA","GBM","HNSC","KICH","KIRC","KIRP","LAML","LGG","LIHC","LUAD","LUSC","MESO","PAAD","PCPG","READ","SARC","SKCM","STAD","THCA","THYM","UVM")){
      exp <- read.delim(paste0("expression/expression_tcga_",cancer,".txt"), check.names = FALSE)
      panda_results <- panda(motif,exp, ppi, mode="intersection")
      write.csv(panda_results@regNet, file=paste0("../data/network/panda_", cancer, ".csv"), quote=F)
    }
}

These networks can be downloaded and visualized in GRAND database in the cancer seciotn (https://grand.networkmedicine.org/cancers).
Because we used the same generic input networks to PANDA across 26 cancer types and used the same normalization steps for gene expression, it is easier to compare regualtory changes between cancer types in comparison to networks that were generated with different computational parameters.
In GRAND, we generated a "normal" matching set for these cancer types by using gene expression form GTEx and the same generic priors as well, to built an online comparison tool for differential analysis within and between tissues: https://grand.networkmedicine.org/netcomp/aggregate/1/

For example, we can load the UVM PANDA network:

In [ ]:
regNet <- read.csv(paste0(ppath,"panda_UVM.csv"), row.names=1)
regNet

It has 644 TFs and the set of genes has been reduced to 29,374 from the initial 60,483 in gene expression data as expected by setting the `mode` to `intersection` in PANDA.

## References
1 - Glass, Kimberly, et al. "Passing messages between biological networks to refine predicted interactions." PloS one 8.5 (2013): e64832.

2 - Hicks, Stephanie C., et al. "Smooth quantile normalization." Biostatistics 19.2 (2018): 185-198.

3 - Szklarczyk, Damian, et al. "The STRING database in 2017: quality-controlled protein–protein association networks, made broadly accessible." Nucleic acids research (2016): gkw937.

4 - Weirauch, Matthew T., et al. "Determination and inference of eukaryotic transcription factor sequence specificity." Cell 158.6 (2014): 1431-1443.

5- Tomczak, Katarzyna, Patrycja Czerwińska, and Maciej Wiznerowicz. "The Cancer Genome Atlas (TCGA): an immeasurable source of knowledge." Contemporary oncology 19.1A (2015): A68.

6- Ben Guebila, Marouen, et al. "GRAND: A database of gene regulatory network models across human conditions." bioRxiv (2021).